In [1]:
'''
3D残差网络，出处是《Deep Residual Learning for Image Recognition》，经常用于图像分类，这里先试试它回归任务的表现——效果不好。

在深度神经网络中，随着网络层数的增加，梯度消失和表示瓶颈问题会越来越严重，导致网络性能下降。
为了解决这个问题，ResNet引入了一种称为“残差块”的结构。
残差块通过引入一个“shortcut connection”将输入直接传递到输出，使网络能够学习输入与输出之间的残差，从而减轻深层网络的训练难度。

网络结构：3D卷积-bn-max池化-残差块迭代-avg池化-fc-输出
'''

'\n3D残差网络，出处是《Deep Residual Learning for Image Recognition》。\n在深度神经网络中，随着网络层数的增加，梯度消失和表示瓶颈问题会越来越严重，导致网络性能下降。\n为了解决这个问题，ResNet引入了一种称为“残差块”的结构。\n残差块通过引入一个“shortcut connection”将输入直接传递到输出，使网络能够学习输入与输出之间的残差，从而减轻深层网络的训练难度。\n网络结构：3D卷积-池化-3D卷积-池化-3D卷积*2-池化-3D卷积*2-池化-扁平化-MLP-输出\n'

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import trimesh
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [3]:
# 加载和归一化模型
def load_mesh(file_path):
    mesh = trimesh.load(file_path)
    # 如果加载的是Scene对象，转换为Trimesh对象
    if isinstance(mesh, trimesh.Scene):
        mesh = mesh.dump(concatenate=True)
    return mesh

def normalize_mesh(mesh):
    # 将网格中心平移到原点
    mesh.apply_translation(-mesh.centroid)
    # 计算缩放比例
    scale_factor = 1.0 / max(mesh.extents)
    # 缩放网格
    mesh.apply_scale(scale_factor)
    return mesh

def mesh_to_voxel(mesh, voxel_size=32):
    # 将网格体素化
    voxelized_mesh = mesh.voxelized(pitch=1.0 / voxel_size)
    # 确保体素矩阵的形状为 (voxel_size, voxel_size, voxel_size)
    voxel_matrix = voxelized_mesh.matrix
    padded_matrix = np.zeros((voxel_size, voxel_size, voxel_size), dtype=voxel_matrix.dtype)
    shape = np.minimum(voxel_matrix.shape, (voxel_size, voxel_size, voxel_size))
    padded_matrix[:shape[0], :shape[1], :shape[2]] = voxel_matrix[:shape[0], :shape[1], :shape[2]]
    return padded_matrix

# 读取评分
def read_scores(score_file_path):
    indices = []
    scores = []
    with open(score_file_path, 'r') as f:
        for line in f:
            index, score = line.strip().split()
            indices.append(int(index))
            scores.append(float(score))
    return indices, scores  

# 定义数据集
class TeethDataset(Dataset):
    def __init__(self, score_file_path, voxel_size=32):
        self.indices, self.scores = read_scores(score_file_path)
        self.voxel_size = voxel_size

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        index = self.indices[idx]
        score = self.scores[idx]
        mesh_path = f'{index}.obj'
        mesh = normalize_mesh(load_mesh(mesh_path))
        voxel_data = mesh_to_voxel(mesh, self.voxel_size)
        return torch.tensor(voxel_data[np.newaxis, :], dtype=torch.float32), torch.tensor([score], dtype=torch.float32)

# 创建数据集
score_file_path = 'scores.txt'
dataset = TeethDataset(score_file_path, voxel_size=32)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [4]:
# 定义残差块
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm3d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm3d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv3d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm3d(out_channels)
            )

    def forward(self, x):
        residual = self.downsample(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

# 定义3D ResNet模型
class ResNet3D(nn.Module):
    def __init__(self, block, layers, num_classes=1):
        super(ResNet3D, self).__init__()
        self.in_channels = 32
        self.conv1 = nn.Conv3d(1, 32, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm3d(32)
        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 32, layers[0])
        self.layer2 = self._make_layer(block, 64, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
# 创建模型
model = ResNet3D(ResidualBlock, [2, 2, 2, 2])
criterion = nn.MSELoss()  # 损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001)  # 学习率和优化器

# 训练模型
num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader)}")

# 模型评估
model.eval()
test_loss = 0.0
all_targets = []
all_outputs = []
with torch.no_grad():
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()
        all_targets.extend(targets.numpy().flatten())
        all_outputs.extend(outputs.numpy().flatten())
print(f"Test Loss: {test_loss / len(dataloader)}")

Epoch 1/30, Loss: 4070.2587183902137
Epoch 2/30, Loss: 2629.247304815995
Epoch 3/30, Loss: 1653.0785313656456
Epoch 4/30, Loss: 810.3492544073807
Epoch 5/30, Loss: 527.3399304841694
Epoch 6/30, Loss: 338.13530058609814
Epoch 7/30, Loss: 367.28619716041965
Epoch 8/30, Loss: 260.64283795105786
Epoch 9/30, Loss: 266.83170348719545
Epoch 10/30, Loss: 278.61329148945055
Epoch 11/30, Loss: 258.4184074778306
Epoch 12/30, Loss: 218.5042839050293
Epoch 13/30, Loss: 262.58168051117343
Epoch 14/30, Loss: 233.7589716660349
Epoch 15/30, Loss: 237.899203225186
Epoch 16/30, Loss: 253.9292561380487
Epoch 17/30, Loss: 263.35633789865597
Epoch 18/30, Loss: 227.38986647756477
Epoch 19/30, Loss: 210.73089604628714
Epoch 20/30, Loss: 214.0749441448011
Epoch 21/30, Loss: 200.50351102728592
Epoch 22/30, Loss: 200.4416517960398
Epoch 23/30, Loss: 193.75983509264495
Epoch 24/30, Loss: 215.08609309949372
Epoch 25/30, Loss: 172.23007980145906
Epoch 26/30, Loss: 170.0415783555884
Epoch 27/30, Loss: 170.2491541912

In [6]:
# 计算评价指标
mse = mean_squared_error(all_targets, all_outputs)
rmse = np.sqrt(mse)
mae = mean_absolute_error(all_targets, all_outputs)
r2 = r2_score(all_targets, all_outputs)
mape = np.mean(np.abs((np.array(all_targets) - np.array(all_outputs)) / np.array(all_targets))) * 100

n = len(all_targets)
k = sum(p.numel() for p in model.parameters())
aic = n * np.log(mse) + 2 * k
bic = n * np.log(mse) + k * np.log(n)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R^2: {r2}")
print(f"MAPE: {mape}%")
print(f"AIC: {aic}")
print(f"BIC: {bic}")

MSE: 119.28743743896484
RMSE: 10.921878814697266
MAE: 8.541479110717773
R^2: 0.5327260350993321
MAPE: 14.500020444393158%
AIC: 16601807.230415344
BIC: 41591720.96764955


In [ ]:
'''
最初尝试了25轮训练，但结果不好，30-70段分数严重偏高，最终loss也没降到200以下。
随后尝试了30轮训练,21轮开始loss降到200，25轮降到170，模型稳定性存疑。结果变好了。
'''

In [7]:
# 对150个预备体进行评分
model.eval()
scores = []
with torch.no_grad():
    for i in range(len(dataset)):
        inputs, _ = dataset[i]  # 忽略目标评分，只取输入数据
        inputs = inputs.unsqueeze(0)  # 添加批次维度
        output = model(inputs)
        scores.append(output.item())

# 输出150个预备体的评分
_, Real_Scores = read_scores(score_file_path)

for i, score in enumerate(scores):
    print(f'Tooth {i+1}: Score = {score}', f'Err = {score-Real_Scores[i]}' )

Tooth 1: Score = 58.573482513427734 Err = -16.426517486572266
Tooth 2: Score = 80.21920013427734 Err = 12.219200134277344
Tooth 3: Score = 61.502403259277344 Err = -8.497596740722656
Tooth 4: Score = 95.79093933105469 Err = 26.790939331054688
Tooth 5: Score = 81.233154296875 Err = 5.233154296875
Tooth 6: Score = 53.012699127197266 Err = -10.987300872802734
Tooth 7: Score = 62.81429672241211 Err = -6.185703277587891
Tooth 8: Score = 68.82127380371094 Err = -4.1787261962890625
Tooth 9: Score = 67.59912109375 Err = -9.40087890625
Tooth 10: Score = 67.8121109008789 Err = -9.187889099121094
Tooth 11: Score = 48.268707275390625 Err = -5.731292724609375
Tooth 12: Score = 59.05068588256836 Err = -2.9493141174316406
Tooth 13: Score = 48.581214904785156 Err = -10.418785095214844
Tooth 14: Score = 52.66676712036133 Err = -10.333232879638672
Tooth 15: Score = 53.6026496887207 Err = -3.397350311279297
Tooth 16: Score = 46.344383239746094 Err = -1.6556167602539062
Tooth 17: Score = 57.89688873291015